# Consumption of Banxico API and creation of a dataset

## Libraries

Shot out to the user @EliasManJ who created the library for Banxico API for Python

In [2]:
import pandas as pd
import os
import json
from banxicoapi import banxico_api

## API call and save of the results

In [3]:
api_token = os.environ['BANXICO_TOKEN']
api = banxico_api.BanxicoApi(api_token)

series = ["SP30577", 	# National Consumer Price Index
			"SF29652",	# Monetary base
			"SF283",	# Interest rate for public debt instruments (CETES) 28 days
			"SF31991", 	# International reserves of the Bank of Mexico (USD)
			"SG41"		# Public sector balance
			]
data = api.get(series)

# trasnform to json
with open("api_call.json", "w") as f:
	json.dump(data, f)

## Creation of the dataset from the json file

In [4]:
def clean_column_name(series_id, title):
    """Create a clean column name from series ID and title."""
    clean_title = title.replace('í', 'i').replace('é', 'e').replace('á', 'a').replace('ó', 'o').replace('ú', 'u')
    return f"{series_id}_{clean_title}"

def convert_json_to_csv(json_data, output_dir='output'):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Parse JSON data
    data = json.loads(json_data)
    
    # Dictionary to store all series data
    all_dates = set()
    series_data = {}
    
    # First pass: collect all dates and prepare series data
    for series in data:
        series_id = series['idSerie']
        series_title = series['titulo']
        column_name = clean_column_name(series_id, series_title)
        
        # Create a dictionary for this series
        date_value_dict = {}
        
        for entry in series['datos']:
            try:
                date = pd.to_datetime(entry['fecha'], format='%d/%m/%Y')
                value = entry['dato']
                
                # Handle numeric values with comma as thousand separator
                if isinstance(value, str):
                    if value == 'N/E':
                        value = pd.NA
                    else:
                        # Remove commas and convert to float
                        value = float(value.replace(',', ''))
                
                date_value_dict[date] = value
                all_dates.add(date)
                
            except Exception as e:
                print(f"Error processing entry {entry}: {str(e)}")
        
        series_data[column_name] = date_value_dict
            
        print(f"Processed values for {column_name}: {len(date_value_dict)}")
    
    # Convert to DataFrame
    all_dates = sorted(list(all_dates))
    df_dict = {}
    
    for column_name, date_value_dict in series_data.items():
        # Create a series with all dates, filling missing values with NA
        series_values = [date_value_dict.get(date, pd.NA) for date in all_dates]
        df_dict[column_name] = series_values
    
    # Create final DataFrame
    combined_df = pd.DataFrame(df_dict, index=all_dates)
    
    # Sort by date
    combined_df.sort_index(inplace=True)
    
    # Save to CSV
    output_path = os.path.join(output_dir, 'Inflation_Rate_Variables_Dataset.csv')
    combined_df.to_csv(output_path, encoding='utf-8')
    print(f"\nCreated: {output_path}")

# Load JSON data
with open("api_call.json", "r") as f:
    json_data = f.read()

# Convert JSON to CSV
convert_json_to_csv(json_data)

Processed values for SF29652_Base Monetaria: 359
Processed values for SF31991_Banco de Mexico, Recursos en moneda extranjera, Reserva Internacional (Definida de acuerdo con la Ley del Banco de Mexico de Abril de 1994): 479
Processed values for SP30577_Índice Nacional de Precios al consumidor Variacion mensual: 671
Processed values for SG41_Ingresos y Gastos Presupuestales del Sector Publico Medicion por Ingreso-Gasto, Flujos de Caja Balance publico Balance presupuestario: 575
Processed values for SF283_TIIE a 28 dias Tasa de interes promedio mensual, en por ciento anual: 359

Created: output\Inflation_Rate_Variables_Dataset.csv
